In [1]:
import torch
import os
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,  ConfusionMatrixDisplay

In [2]:
# print(f'\nAvailable cuda = {torch.cuda.is_available()}')

# print(f'\nGPUs availables = {torch.cuda.device_count()}')

In [3]:
# import tensorflow as tf
# from tensorflow.python.client import device_lib

# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# device_lib.list_local_devices()

In [4]:
###########################
# Importar datos benignos #
carpeta_transformados_benign = './Dataset/TransformadosBenign'

# Obtener la lista de archivos pickle en la carpeta
pickle_files = [f for f in os.listdir(carpeta_transformados_benign) if f.endswith('.pickle')]

# Crear una lista vacía para almacenar los dataframes cargados
df_list = []

# Cargar cada archivo pickle en la lista
for file in pickle_files:
    with open(os.path.join(carpeta_transformados_benign, file), 'rb') as f:
        df = pickle.load(f)
        df_list.append(df)

# Combinar los dataframes cargados en un solo dataframe
df = pd.concat(df_list, ignore_index=True)

In [5]:
###########################
# Importar datos malignos #
carpeta_transformados_malware = './Dataset/TransformadosMalware'

# Obtener la lista de archivos pickle en la carpeta
pickle_files = [f for f in os.listdir(carpeta_transformados_malware) if f.endswith('.pickle')]

# Crear una lista vacía para almacenar los dataframes cargados
df_doh_list = []

# Cargar cada archivo pickle en la lista
for file in pickle_files:
    with open(os.path.join(carpeta_transformados_malware, file), 'rb') as f:
        df_doh = pickle.load(f)
        df_doh_list.append(df_doh)

# Combinar los dataframes cargados en un solo dataframe
df_doh = pd.concat(df_doh_list, ignore_index=True)

In [6]:
del df_list
del df_doh_list

In [7]:
print("Benign data shape: " + str(df.shape))
print("Malware data shape: " + str(df_doh.shape))

Benign data shape: (287007, 1)
Malware data shape: (150144, 1)


In [8]:
# Cut data
# df = df.iloc[:124000]
# df_doh = df_doh.iloc[:70000]

In [9]:
df['label'] = np.zeros(len(df))
df_doh['label'] = np.ones(len(df_doh))

In [10]:
data, data_doh = df, df_doh
print(data.memory_usage(index=True, deep=True))
print(data_doh.memory_usage(index=True, deep=True))

Index                           128
udps.n_bytes_per_packet    42878928
label                       2296056
dtype: int64
Index                           128
udps.n_bytes_per_packet    29498240
label                       1201152
dtype: int64


In [11]:
del df
del df_doh

In [12]:
full_data = pd.concat([data.sample(frac=0.3), data_doh.sample(frac=0.3)]).sample(frac=1)

In [ ]:
del data
del data_doh

In [13]:
full_data = full_data.explode('udps.n_bytes_per_packet', ignore_index=True)

In [14]:
X = np.array(full_data['udps.n_bytes_per_packet'].tolist())
y = np.array(full_data['label'])

In [ ]:
del full_data

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

MemoryError: Unable to allocate 1.47 GiB for an array with shape (384559, 1024) and data type int32

In [ ]:
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo: {:.2f}".format(accuracy))

Precisión del modelo: 0.91


In [ ]:
cm = confusion_matrix(y_test, y_pred)

In [ ]:
print("Matriz de Confusión:")
print(cm)

Matriz de Confusión:
[[ 1531  2005]
 [ 4038 58185]]


In [ ]:
filename = "naive_bayes_v1.pkl"
with open(filename, 'wb') as file:
    pickle.dump(model, file)